In [ ]:
# setup

import os
import re
import json
import time
import csv

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.common.exceptions import TimeoutException

In [ ]:
def wait_until_present(driver, time, *locator):
    return WebDriverWait(driver, time).until(expected_conditions.presence_of_element_located(locator))

In [ ]:
# if want actual chrome driver

driver = webdriver.Chrome(executable_path = os.getcwd() + "/chromedriver")

In [ ]:
# if want headless chrome driver

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1200x600')
driver = webdriver.Chrome(executable_path = os.getcwd() + "/chromedriver", options=options)

In [ ]:
# set to specific year, go to correct link.

year = 1943 
driver.get("https://yvng.yadvashem.org/index.html?language=en&advancedSearch=true&yd_value={}&yd_type=Exact&pld_value=Auschwitz&pld_type=synonyms&fate_value=murdered".format(year))

In [ ]:
# scrape data 
data
maxp = driver.find_elements_by_class_name("spn-pages-total")[2].text # total number of pages for specified year
data = [dict(birth = 0, location = 0) for x in range(int(maxp)*50+2)] # creates an array of dicts to store data
p = 0 # initializes page number
for page in range(int(maxp)):
    print(p, end = ', ') # prints page number
    # checks for "..." loading graphic
    WebDriverWait(driver, 15).until(lambda driver: driver.find_element_by_xpath('//*[@id="overlay"][contains(@style, "display: none")]'))
    # closes out of popup ad if present
    if len(driver.find_elements_by_xpath('//*[@aria-hidden="true"]')) == 54:
        driver.find_elements_by_xpath('//*[@class="close modal_close"]')[1].click()
    results = wait_until_present(driver, 10, By.XPATH, '//*[@role="rowgroup"]')
    info = results.find_elements_by_xpath('//*[@role="gridcell"]')
    for i in range(int(len(results.find_elements_by_xpath('//*[@role="gridcell"]'))/12)):
        data[p*50+i] = {'birth': info[2 + 12*i].text, 'location': info[3 + 12*i].text}
    driver.find_elements_by_id("next-page-u")[1].click()
    p+=1

In [ ]:
# export data as csv

with open('Auschwitz{}.txt'.format(year), 'w') as outfile:
    writer = csv.DictWriter(outfile, fieldnames=["birth", "location"])
    writer.writeheader()
    writer.writerows(data)
    outfile.close()